<img src="http://oci02.img.iteso.mx/identidad_de_instancia_2018/ITESO/Logos%20ITESO/Logo-ITESO-Principal.jpg">

<center> <font color= #004A94> <font size = 6> Msc Ciencia de datos </font> <br> <br> <font color= #047CFB> <font size = 5>I.F. Juan Francisco Muñoz Elguezabal</font> <br> <br>

<center><font color= #42c29b> <font size = 6> Análisis Estadístico Multivariable </font> <br> <br> <font color= #047CFB> <font size = 5> Exámen parcial 2 </font>

---

### Cargar Librerias

In [1]:
import pandas as pd                                       # dataframes y utilidades
import numpy as np                                        # operaciones matematicas
import statsmodels.api as sm                              # modelos estadisticos: anova
import visualizaciones as vs                              # importar visualizaciones desde otro script
import plotly.express as px                               # visualizaciones express con plotly
from statsmodels.stats.multicomp import MultiComparison   # herramientas estadisticas: multicomparacion
from statsmodels.formula.api import ols                   # herramientas estadisticas: modelo lineal con ols
from sklearn.model_selection import train_test_split      # separar conjunto de entrenamiento y prueba

pd.set_option('display.max_rows', None)                   # sin limite de renglones maximos para mostrar pandas
pd.set_option('display.max_columns', None)                # sin limite de columnas maximas para mostrar pandas
pd.set_option('display.width', None)                      # sin limite el ancho del display
pd.set_option('display.expand_frame_repr', False)         # visualizar todas las columnas de un dataframe
pd.options.mode.chained_assignment = None                 # para evitar el warning enfadoso de indexacion

## Ejercicio 1 (Calidad del aire)

### A) ¿Qué hipótesis se deben de cumplir para poder obtener un modelo que describa la relación lineal entre dos variables?

- Linealidad (Entre las variables explicativas)
- Independencia (Entre las variables y también entre los residuales)
- Normalidad en la distribución de los valores (En los residuales)
- Homocedasticidad en la varianza entre los valores(En los residuales)

### B) Obtenga un modelo lineal simple que permita estimar la concentración de emisiones contaminantes y la temperatura de la región. Escriba la ecuación del modelo e interprete los resultados obtenidos.

In [2]:
df_datos = pd.read_excel('datos/AirQuality.xlsx')

x_train, x_test, y_train, y_test = train_test_split(df_datos.loc[:, 'T'],
                                                    df_datos.loc[:, 'CO(GT)'],
                                                    test_size=0.10, random_state=0, shuffle=False)

In [3]:
# Agregar interceptos a X en entrenamiento y prueba
x_train_betha = sm.add_constant(x_train)
x_test_betha = sm.add_constant(x_test)
modelo = sm.OLS(y_train, x_train_betha, hasconst=False)
resultados_a = modelo.fit()
print(resultados_a.summary())

                                 OLS Regression Results                                
Dep. Variable:                 CO(GT)   R-squared (uncentered):                   0.745
Model:                            OLS   Adj. R-squared (uncentered):              0.744
Method:                 Least Squares   F-statistic:                              655.4
Date:                Wed, 23 Oct 2019   Prob (F-statistic):                   9.12e-134
Time:                        22:11:30   Log-Likelihood:                         -771.78
No. Observations:                 450   AIC:                                      1548.
Df Residuals:                     448   BIC:                                      1556.
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

| Parámetro | Coeficiente $\beta_i$  | p_value Vs $\alpha$      | Conclusión 1        |
| --------- | ---------------------- | ------------------------ | ------------------- |
| const     | 0.7758                 | 0.001 < 0.05             | Si es significativo |
| T         | 0.1067                 | 0.000 < 0.05             | Si es significativo | 

<center> La ecuación del modelo es: $y = \beta_0 + \beta_1 x_i => y = 0.7758 + 0.1067 x_i$ </center>

El valor de *R-squared*, que fue de **0.745** es mayor a **0.5** y se acerca aceptablemente a 1, por lo tanto este modelo aunque es simple si resulta ajustarse lo suficiente a los datos para ser utilizado con cierta confianza. También, tanto el *Intercepto* como la *T (Temperatura)* son componentes que si tienen poder explicativo dentro del modelo, esto debido a que su valor *p_value*, para ambos casos, fue menor 0.05. Se puede concluir también que la relación entre las particulas contaminantes y la temperatura ambiente tienen una relación directa y estadísticamente linealmente proporcional.

### C) Indique si el modelo lineal se ajusta a los datos basado en las predicciones obtenidas. Justifica tu respuesta.

In [4]:
resultados_p = resultados_a.predict(np.array(x_test_betha))
df_resultados_a = pd.DataFrame({'CO(GT)_real': y_test,
                                'CO(GT)_pred': np.round(resultados_p, 2),
                                'Diff': np.round(resultados_p-y_test, 2)})
df_resultados_a.head()

,CO(GT)_real,CO(GT)_pred,Diff
450,1.4,2.47,1.07
451,1.6,2.57,0.97
452,1.5,2.62,1.12
453,1.4,2.65,1.25
454,1.5,2.63,1.13


In [5]:
np.round((df_resultados_a['Diff']**2).mean(), 2)

1.25

Se puede observar en los datos de prueba que el ajuste del modelo no es perfecto, sin embargo, las diferencias entre los valores conocidos y los ajustados son menores a 1, por lo que pueden ser útiles. 

### D) Además de la temperatura, ¿qué otras variables podrían influir en la concentración de emisiones contaminantes?

In [6]:
df_datos.head()

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,2004-03-10,18:00:00,2.6,1360.00,150,11.881723,1045.50,166,1056.25,113,1692.00,1267.50,13.60,48.875001,0.757754
1,2004-03-10,19:00:00,2.0,1292.25,112,9.397165,954.75,103,1173.75,92,1558.75,972.25,13.30,47.700000,0.725487
2,2004-03-10,20:00:00,2.2,1402.00,88,8.997817,939.25,131,1140.00,114,1554.50,1074.00,11.90,53.975000,0.750239
3,2004-03-10,21:00:00,2.2,1375.50,80,9.228796,948.25,172,1092.00,122,1583.75,1203.25,11.00,60.000000,0.786713
4,2004-03-10,22:00:00,1.6,1272.25,51,6.518224,835.50,131,1205.00,116,1490.00,1110.00,11.15,59.575001,0.788794


In [7]:
df_datos.corr()

,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
CO(GT),1.000000,0.915981,0.689655,0.962229,0.948724,0.942809,-0.820789,0.817970,0.929510,0.865410,0.333028,-0.229544,0.128218
PT08.S1(CO),0.915981,1.000000,0.546282,0.923572,0.930259,0.903731,-0.832460,0.807018,0.944736,0.927153,0.382201,-0.172571,0.301298
NMHC(GT),0.689655,0.546282,1.000000,0.689937,0.669632,0.652279,-0.565793,0.529758,0.601316,0.578188,0.134781,-0.202458,-0.161529
C6H6(GT),0.962229,0.923572,0.689937,1.000000,0.982491,0.925228,-0.843450,0.804037,0.961651,0.889128,0.393216,-0.273475,0.131209
PT08.S2(NMHC),0.948724,0.930259,0.669632,0.982491,1.000000,0.926313,-0.913453,0.848311,0.954386,0.898204,0.427983,-0.304458,0.129139
NOx(GT),0.942809,0.903731,0.652279,0.925228,0.926313,1.000000,-0.822642,0.884117,0.909803,0.879669,0.295992,-0.182456,0.137797
PT08.S3(NOx),-0.820789,-0.832460,-0.565793,-0.843450,-0.913453,-0.822642,1.000000,-0.811240,-0.854398,-0.838706,-0.421554,0.276580,-0.201624
NO2(GT),0.817970,0.807018,0.529758,0.804037,0.848311,0.884117,-0.811240,1.000000,0.772647,0.790089,0.416462,-0.319800,0.083196
PT08.S4(NO2),0.929510,0.944736,0.601316,0.961651,0.954386,0.909803,-0.854398,0.772647,1.000000,0.900994,0.356502,-0.111779,0.364037
PT08.S5(O3),0.865410,0.927153,0.578188,0.889128,0.898204,0.879669,-0.838706,0.790089,0.900994,1.000000,0.329788,-0.146415,0.270543


Para tener una idea de que variables podrían ser útiles de utilizar en el ajuste de algún modelo explicativo, se puede calcular la matríz de correlaciones y obtener aquellas variables que tengan una correlación mayor a 0.5, ya sea positiva (correlación > +0.5) o negativa (correlación < -0.5). 

In [8]:
correlaciones = df_datos.corr()
significativas = correlaciones[abs(correlaciones['CO(GT)']) > 0.5]['CO(GT)'][1:]
significativas

PT08.S1(CO)      0.915981
NMHC(GT)         0.689655
C6H6(GT)         0.962229
PT08.S2(NMHC)    0.948724
NOx(GT)          0.942809
PT08.S3(NOx)    -0.820789
NO2(GT)          0.817970
PT08.S4(NO2)     0.929510
PT08.S5(O3)      0.865410
Name: CO(GT), dtype: float64

### E) Obtenga el modelo lineal múltiple que permita estimar esas concentraciones en función de las variables que mencionó en el inciso anterior. Interprete los resultados obtenidos.

In [9]:
vars = list(significativas.index)
vars

['PT08.S1(CO)',
 'NMHC(GT)',
 'C6H6(GT)',
 'PT08.S2(NMHC)',
 'NOx(GT)',
 'PT08.S3(NOx)',
 'NO2(GT)',
 'PT08.S4(NO2)',
 'PT08.S5(O3)']

In [10]:
x_train, x_test, y_train, y_test = train_test_split(df_datos.loc[:, vars],
                                                    df_datos.loc[:, 'CO(GT)'],
                                                    test_size=0.10, random_state=0, shuffle=False)

# Agregar interceptos a X en entrenamiento y prueba
x_train_betha = sm.add_constant(x_train)
x_test_betha = sm.add_constant(x_test)
modelo = sm.OLS(y_train, x_train_betha, hasconst=False)
resultados_b = modelo.fit()
print(resultados_b.summary())

                                 OLS Regression Results                                
Dep. Variable:                 CO(GT)   R-squared (uncentered):                   0.986
Model:                            OLS   Adj. R-squared (uncentered):              0.986
Method:                 Least Squares   F-statistic:                              3104.
Date:                Wed, 23 Oct 2019   Prob (F-statistic):                        0.00
Time:                        22:11:30   Log-Likelihood:                         -118.69
No. Observations:                 450   AIC:                                      257.4
Df Residuals:                     440   BIC:                                      298.5
Df Model:                          10                                                  
Covariance Type:            nonrobust                                                  
                    coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------

Se puede apreciar que de todas las variables propuestas, sólo *'NO2(GT)'* fue la única que resultó ser no significativa (debido a su *p_value*, $1 > 0.05$). Debido a esto, el coeficiente de *R-Squared* pasó, de **0.745** en el modelo que sólo utilizó 1 variable, a **0.986** que utilizó 9 variables.

In [11]:
resultados_p = resultados_b.predict(np.array(x_test_betha))
df_resultados_b = pd.DataFrame({'CO(GT)_real': y_test,
                                'CO(GT)_pred': np.round(resultados_p, 2),
                                'Diff': np.round(resultados_p-y_test, 2)})
df_resultados_b.head()

,CO(GT)_real,CO(GT)_pred,Diff
450,1.4,1.50,0.10
451,1.6,1.65,0.05
452,1.5,1.63,0.13
453,1.4,1.60,0.20
454,1.5,1.65,0.15


In [12]:
np.round((df_resultados_b['Diff']**2).mean(), 2)

0.08

### f) ¿Existe un cambio significativo en las predicciones si se hace la regresión lineal con una o más variables independientes? Justifique su respuesta.

Si existe un cambio y es bastante significativo, el MSE (Error Cuadrático Promedio) del modelo con una variable fue de 1.25, mientras que del modelo con 9 variables se redujo hasta 0.08. Por lo tanto, fue útil, no sólo agregar *mas* variables, sino, agregar aquellas que tuvieran una alta correlación con la variable a explicar

### g) Obtenga los contrastes individuales de los coeficientes $\beta_{i}$ con $\alpha = 0.05$. ¿Qué puede concluir con los resultados obtenidos?

*PT08.S1(CO)*, *NMHC(GT)*, *C6H6(GT)*, *NOx(GT)*, tienen una relación positiva respecto a la variable a explicar, mientras que *PT08.S2(NMHC)*, *PT08.S3(NOx)*, *PT08.S4(NO2)*, *PT08.S5(O3)* tienen una relación negativa. Y para el caso de *NO2(GT)*, su *p_value* resultó mayor a 0.05 por lo tanto se elimina como variable significativa en esta configuración de modelo.

#### f) Estime la concentración de emisiones contaminantes si la temperatura es de 27 y la humedad relativa es del 50%  y la humedad absoluta de 0.7?

## Ejercicio 2 (Huachicoleo)

Se quiere saber si hay una variación  en el precio de la gasolina y el diésel por entidad federativa en la República Mexicana. Para realizar este estudio se recomienda utilizar el análisis de la varianza.

In [13]:
df_datos = pd.read_excel('datos/gasolina.xlsx')

In [14]:
df_datos.head()

,Entidad_federativa,Gasolina_91_octanos,Gasolina_87_octanos,Diesel,Anio_reporte
0,Aguascalientes,17.95,16.19,17.28,2017
1,Aguascalientes,17.95,16.19,17.28,2017
2,Aguascalientes,17.86,16.11,17.16,2017
3,Aguascalientes,17.85,16.12,17.10,2017
4,Aguascalientes,17.78,16.04,16.97,2017


In [15]:
Estados = list(set(df_datos['Entidad_federativa']))
Estados

['Durango',
 'Tlaxcala',
 'Chiapas',
 'Zacatecas',
 'Jalisco',
 'Tamaulipas',
 'Nayarit',
 'Campeche',
 'Hidalgo',
 'Guanajuato',
 'Chihuahua',
 'Guerrero',
 'Oaxaca',
 'Queretaro',
 'Veracruz de Ignacio de la Llave',
 'Morelos',
 'Nuevo Leon',
 'Aguascalientes',
 'Coahuila de Zaragoza',
 'Yucatan',
 'Tabasco',
 'Quintana Roo',
 'Sinaloa',
 'Baja California Sur',
 'Baja California',
 'Colima',
 'Michoacan de Ocampo',
 'Puebla',
 'Durango Gomez Palacio',
 'Sonora',
 'Mexico',
 'Ciudad de Mexico',
 'San Luis Potosi']

In [16]:
len(Estados)

33

### A) ¿Qué tipo de ANOVA está utilizando?

Un ANOVA de un sólo sentido o (ONE-WAY ANOVA)

### B) ¿Cuál sería la justificación para utilizar este método?

Debido a que se quiere encontrar si el precio de la gasolina y del diesel presentan una variación estadísticamente significativa según el estado de la república donde se registraron esos precios. El objetivo es validar o rechazar, con evidencia estadística, las siguientes hipótesis: 

Estadísticamente:

- **H0**: la media de los grupos es la misma.
- **Ha**: la media de al menos un grupo es diferente de la de los demás grupos.

En el contexto del problema: 

- **H0**: Entre todos los estados, no existe variación en el precio de la gasolina y del diesel
- **Ha**: Al menos entre 1 estado y los demás, hubo variación significativa en el precio de la gasolina y del diesel.

### C) ¿En qué escala de medición deben estar las variables?

Para la variable independiente, en este caso la *Entidad Federativa* es del tipo *Nominal* u *Ordinal*, que son variables cualitativas o categóricas. Para el caso de la variable dependiente debe de ser *Continua* (como intervalo o ratio), como lo es el precio de la gasolina y del diesel este caso.

### D) ¿Qué hipótesis se deben de cumplir para utilizar el ANOVA?

- Normalidad en los residuales
- Independencia de observaciones
- Homocedasticidad en los residuales
- Aleatoriedad en la medición de las observaciones

### E) Contrastar si el precio de la gasolina y el diésel tienen variación dependiendo de la entidad federativa. Para realizar el contraste considere un nivel de significancia del  5%

#### Visualizacion de variables

In [17]:
# crear cuadro de datos para modelo ANOVA
df_data_anova = df_datos[['Entidad_federativa', 'Gasolina_91_octanos', 'Gasolina_87_octanos', 'Diesel']]

In [18]:
# ajustar modelo lineal para Gasolina_91_octanos
model_91 = ols('Gasolina_91_octanos ~ C(Entidad_federativa)', data=df_data_anova).fit()
model_91.summary()

# ajustar modelo lineal para Gasolina_87_octanos
model_87 = ols('Gasolina_87_octanos ~ C(Entidad_federativa)', data=df_data_anova).fit()
model_87.summary()

# ajustar modelo lineal para Diesel
model_di = ols('Diesel ~ C(Entidad_federativa)', data=df_data_anova).fit()
model_di.summary()

# tabla anova Gasolina_91_octanos
anova_table_91 = sm.stats.anova_lm(model_91, typ=2)

# tabla anova Gasolina_87_octanos
anova_table_87 = sm.stats.anova_lm(model_87, typ=2)

# tabla anova Diesel
anova_table_di = sm.stats.anova_lm(model_di, typ=2)

### F) De una interpretación de los resultados obtenidos.

In [19]:
anova_table_91

,sum_sq,df,F,PR(>F)
C(Entidad_federativa),151.650551,32.0,2.78364,7.785166e-07
Residual,1346.658373,791.0,NaN,NaN


Se observa que el valor p es **7.785166e-07** y es < 0.05 por lo tanto *se rechaza* la **H0: la media de los grupos es la misma.**. Y se acepta la **Ha:  la media de al menos un grupo es diferente de la de los demás grupos.**. En este contexto, se rechaza que **Entre todos los estados, no existe variación en el precio de la gasolina con 91 de octanaje**. Se acepta la hipótesis que, **Al menos entre 1 estado y los demás, hubo variación significativa en el precio de la gasolina.** Escrito de una manera más científica, existe evidencia suficiente para creer que existe *huachicoleo*

In [20]:
anova_table_87

,sum_sq,df,F,PR(>F)
C(Entidad_federativa),213.997710,32.0,3.16884,1.706444e-08
Residual,1669.303552,791.0,NaN,NaN


Se observa que el valor p es **1.706444e-08** y es < 0.05 por lo tanto *se rechaza* la **H0: la media de los grupos es la misma.**. Y se acepta la **Ha:  la media de al menos un grupo es diferente de la de los demás grupos.** En este contexto, se rechaza la hipótesis que **Entre todos los estados, no existe variación en el precio de la gasolina con 87 de octanaje**. Se acepta la hipótesis que, **Al menos entre 1 estado y los demás, hubo variación significativa en el precio de la gasolina**  Escrito de una manera más científica, existe evidencia suficiente para creer que existe *huachicoleo*

In [21]:
anova_table_di

,sum_sq,df,F,PR(>F)
C(Entidad_federativa),46.584636,32.0,0.613628,0.955267
Residual,1876.565733,791.0,NaN,NaN


Se observa que el valor p es **0.955267** y es > 0.05, por lo tanto *se acepta* la **H0: Entre todos los estados, no existe variación en el precio del diesel**. Por lo tanto, científicamente podemos decir que el *Diesel* ha estado libre del *Huachicoleo*

## Ejercicio 3 (Problemas de sueño)

Para la base de datos cuestionario de "problemas de sueño”
Se quiere analizar si existe alguna relación entre la calidad del sueño y el género. 

In [22]:
df_datos = pd.read_csv('datos/ProblemasSuenio.csv')

In [23]:
df_datos.head()

,folio,inte,idinsert,entidad,munici,locali,maq,edad,sexo,intsel,intsel2,su101,su102,su103,su104,su105,su106,su107,su107a,su107b
0,010010001209A0020131,1,1076190,1,1,1,MC036,67,1,1,NaN,6,2,2,2,NaN,NaN,2,CLONAZEPAM,NaN
1,010010001209A0020161,1,1076193,1,1,1,MC036,55,1,1,10.0,8,1,3,2,NaN,NaN,1,NaN,NaN
2,010010001209A0080071,2,1075466,1,1,1,MC034,44,0,1,NaN,6,2,3,2,NaN,NaN,1,NaN,NaN
3,010010001209A0080101,1,1075538,1,1,1,MC034,47,0,1,NaN,6,2,3,1,1.0,1.0,1,NaN,NaN
4,010010001209A0120041,1,1076278,1,1,1,MC036,36,1,1,NaN,8,1,2,1,1.0,1.0,1,NaN,NaN


In [24]:
len(df_datos)

600

### A ) Elija dos variables que puede utilizar para llevar acabo este análisis. Justifique su respuesta.

- *su101* : por ser un conteo de horas.
- *su107* : por ser una variable que registró frecuencia de uso de algo

In [25]:
datos = df_datos[['su101', 'su107']]
datos.index = df_datos['sexo']
datos.head()

,su101,su107
sexo,,
1,6,2
1,8,1
0,6,1
0,6,1
1,8,1


### B) Obtenga la tabla de logaritmos de las frecuencias.

In [26]:
datos_log = np.log(datos)

In [27]:
datos_log.head()

,su101,su107
sexo,,
1,1.791759,0.693147
1,2.079442,0.000000
0,1.791759,0.000000
0,1.791759,0.000000
1,2.079442,0.000000


### C) Obtenga los valores Mu1_i    

In [28]:
datos_log.loc['media_c'] = datos_log.mean(axis=0)
datos_log.tail()

,su101,su107
sexo,,
0,2.484907,0.000000
0,1.945910,0.000000
0,2.079442,0.000000
0,2.197225,0.000000
media_c,1.972999,0.035813


### D) Obtenga los valores Mu2_j

In [29]:
datos_log['media_f'] = datos_log.mean(axis=1)
datos_log.head()

,su101,su107,media_f
sexo,,,
1,1.791759,0.693147,1.242453
1,2.079442,0.000000,1.039721
0,1.791759,0.000000,0.895880
0,1.791759,0.000000,0.895880
1,2.079442,0.000000,1.039721


### E) De una interpretación de los resultados obtenidos.

In [30]:
# media global
mu = datos_log.iloc[-1,-1]
# media de las filas
mu1 = datos_log.iloc[:,-1]-mu
# media de las columnas
mu2 = datos_log.iloc[-1,:]-mu

media_f es la media de las filas, y media_c es la media de las columnas. Cada uno de estos valores 

In [31]:
datos_rec = np.ones(np.shape(datos_log))*mu
datos_rec

# sumarle a cada valor la media de las filas
for k in np.arange(np.shape(datos_log)[0]):
    datos_rec[k,:] = datos_rec[k,:]+mu2

# sumarle a cada valor la media de las columnas
for k in np.arange(np.shape(datos_log)[1]):
    datos_rec[:,k] = datos_rec[:,k]+mu1

# seleccionar solo los valores originales (sin filas de mu1 ni mu2)
datos_rec = datos_rec[0:-1, 0:-1]
datos_rec

array([[ 2.2110466 ,  0.27386005],
       [ 2.00831405,  0.07112749],
       [ 1.86447301, -0.07271354],
       ...,
       [ 1.94154835,  0.0043618 ],
       [ 2.00831405,  0.07112749],
       [ 2.06720556,  0.13001901]])

In [32]:
tab_mu12 = pd.DataFrame(np.transpose(np.transpose(datos_log-mu-mu2)-mu1))
tab_mu12.iloc[:,-1] = mu1
tab_mu12.iloc[-1,:] = mu2
tab_mu12.head()

,su101,su107,media_f
sexo,,,
1,-0.419287,0.419287,0.238047
1,0.071127,-0.071127,0.035315
0,-0.072714,0.072714,-0.108526
0,-0.072714,0.072714,-0.108526
1,0.071127,-0.071127,0.035315


In [33]:
tab_mu12.iloc[0:-1,0:-1].head()

,su101,su107
sexo,,
1,-0.419287,0.419287
1,0.071127,-0.071127
0,-0.072714,0.072714
0,-0.072714,0.072714
1,0.071127,-0.071127


In [34]:
datos_rec_mod = datos_rec + tab_mu12.iloc[0:-1,0:-1]

In [35]:
datos_rec_mod.head()

,su101,su107
sexo,,
1,1.791759,0.693147
1,2.079442,0.000000
0,1.791759,0.000000
0,1.791759,0.000000
1,2.079442,0.000000


In [36]:
datos_log.head()

,su101,su107,media_f
sexo,,,
1,1.791759,0.693147,1.242453
1,2.079442,0.000000,1.039721
0,1.791759,0.000000,0.895880
0,1.791759,0.000000,0.895880
1,2.079442,0.000000,1.039721


### F) ¿Las variables son independientes o no son independientes? Justifique su respuesta. 

No son independientes. Por que la frecuencia ij no fue el producto de las marginales.

### G) Si las variables no son independientes, obtenga la tabla del efecto de la interacción entre el factor fila y el factor columna. De una interpretación de los resultados obtenidos.

Quedó más arriba.

### H) Por qué se utilizan los modelos log-lineal para analizar este tipo de datos?

Es útil para analizar la relacion entre dos o mas variables categóricas, acomodadas en una tabla de contingencia. Lo característico es que todas las variables que se analizan se consideran como variables respuesta, es decir, no se distingue entre variables independientes y dependientes.

### I) ¿Qué representan los términos del modelo log-lineal para el caso de no independencia

La interacción no capturada entre los términos

### J) Si tuviera tres variables para analizar y estas variables no fueran independientes, ¿cómo quedaría expresado el modelo log-lineal?

ln(nijk) = mu + muif + mujc + mukd + muijcf + muikfd + muijkcfd